In [3]:
import helper_functions
import numpy as np
#Import keras from tensorflow and not as a standalone thing (that makes things work weird idk why)
from tensorflow import keras
# CNN
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, Conv1D, Activation, Conv2D, MaxPooling2D, Input
from tensorflow.keras.layers import InputLayer, Dropout, LSTM, BatchNormalization, Flatten, ZeroPadding1D, AveragePooling2D, BatchNormalization, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D
import pickle
#import helper_functions
from tensorflow.keras.layers import GlobalMaxPooling1D, Dropout, BatchNormalization


In [4]:
data_path = 'data'
with open(data_path, 'rb') as f:
    data = pickle.Unpickler(f).load()

In [5]:
# Shuffle the data
dataToShuffle = []

for log, mfcc, label in zip(data['log-mel'],data['mfccs'],data['classes']):
    dataToShuffle.append([log,mfcc,label])

ShuffledData = np.array(dataToShuffle)
np.random.shuffle(ShuffledData)

In [6]:
# Separate the data into training and test datasets
Train = ShuffledData[:45305]
Test = ShuffledData[45305:]

log_train = []
mfcc_train = []
y_train = []

log_test = []
mfcc_test = []
y_test = []

for log, mfcc, label in Train:
    log_train.append(log)
    mfcc_train.append(mfcc)
    y_train.append(label)
    
for log, mfcc, label in Test:
    log_test.append(log)
    mfcc_test.append(mfcc)
    y_test.append(label)


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

for i in range(2):
    label = y_train[45000+i]
    plt.figure()
    plt.title("Class id: "+ str(label))
    plt.imshow(x_train[45000+i])

In [56]:
log_test[0].shape


(15, 173)

In [24]:
InputShape = log_test[0].shape
output_dim = y_test[0].shape[0]

dropout = 0.2

#based on model 1 from https://www.researchgate.net/publication/348432098_Speech_recognition_based_on_Convolutional_neural_networks_and_MFCC_algorithm
# Getting around 80% accuracy
model = Sequential()
model.add(InputLayer(input_shape=InputShape))
model.add(ZeroPadding1D(padding=2))
model.add(Conv1D(filters=256, kernel_size=10))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Conv1D(filters=256, kernel_size=5))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
#model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dense(output_dim))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [25]:
#Train the model 
x_train = np.array(mfcc_train)
y_train = np.array(y_train)
model.fit(x=x_train, y=y_train, epochs=12, batch_size=128)

Epoch 1/12
354/354 [==============================] - 14s 38ms/step - loss: 25.5278 - accuracy: 0.5061
Epoch 2/12
354/354 [==============================] - 12s 35ms/step - loss: 6.7796 - accuracy: 0.6083
Epoch 3/12
354/354 [==============================] - 14s 40ms/step - loss: 4.5208 - accuracy: 0.6680
Epoch 4/12
354/354 [==============================] - 14s 40ms/step - loss: 3.5754 - accuracy: 0.7010
Epoch 5/12
354/354 [==============================] - 14s 41ms/step - loss: 3.0088 - accuracy: 0.7276
Epoch 6/12
354/354 [==============================] - 13s 37ms/step - loss: 2.6306 - accuracy: 0.7465
Epoch 7/12
354/354 [==============================] - 14s 40ms/step - loss: 2.3186 - accuracy: 0.7629
Epoch 8/12
354/354 [==============================] - 14s 40ms/step - loss: 2.0775 - accuracy: 0.7757
Epoch 9/12
354/354 [==============================] - 14s 39ms/step - loss: 1.9161 - accuracy: 0.7917
Epoch 10/12
354/354 [==============================] - 14s 39ms/step - loss: 1.74

In [26]:
#based on https://towardsdatascience.com/tensorflow-speech-recognition-challenge-solution-outline-9c42dbd219c9
#this one could be quite good, ^^ was getting like 80-85% accuracy (It's the 1D CNN)
#was getting barely over 60% but made the model less deep (as can be seen by comments) and it got much better ~75%
model2 = Sequential()

model2.add(BatchNormalization())

model2.add(Conv1D(64,kernel_size=2,padding='same'))
model2.add(Activation('relu'))
model2.add(Conv1D(64,kernel_size=2,padding='same'))
model2.add(Activation('relu'))

model2.add(Dropout(0.2))
#model.add(MaxPooling1D(2))

#model.add(Conv1D(128, 3,padding='same'))
#model.add(Activation('relu'))
#model.add(Conv1D(128, 3,padding='same'))
#model.add(Activation('relu'))

#model.add(Dropout(0.2))
#model.add(MaxPooling1D(2))

#model.add(Conv1D(128, 3,padding='same'))
#model.add(Activation('relu'))
#model.add(Conv1D(128, 3,padding='same'))
#model.add(Activation('relu'))

#model.add(Dropout(0.2))
#model.add(MaxPooling1D(2))

model2.add(Conv1D(128, 3,padding='same'))
model2.add(Activation('relu'))
model2.add(Conv1D(128, 3,padding='same'))
model2.add(Activation('relu'))


model2.add(Dropout(0.2))
model2.add(MaxPooling1D(2))

model2.add(GlobalMaxPooling1D())

model2.add(Dense(12, activation="softmax"))
    

In [27]:

opt = keras.optimizers.Adam(learning_rate=0.01)

model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [28]:
#Train the model 
x_train = np.array(mfcc_train)
y_train = np.array(y_train)
model2.fit(x=x_train, y=y_train, epochs=12, batch_size=128)

Epoch 1/12
354/354 [==============================] - 10s 27ms/step - loss: 1.3650 - accuracy: 0.6341
Epoch 2/12
354/354 [==============================] - 10s 27ms/step - loss: 1.0586 - accuracy: 0.6729
Epoch 3/12
354/354 [==============================] - 10s 28ms/step - loss: 0.9508 - accuracy: 0.7005
Epoch 4/12
354/354 [==============================] - 9s 25ms/step - loss: 0.9067 - accuracy: 0.7125
Epoch 5/12
354/354 [==============================] - 9s 25ms/step - loss: 0.8781 - accuracy: 0.7213
Epoch 6/12
354/354 [==============================] - 9s 25ms/step - loss: 0.8549 - accuracy: 0.7289
Epoch 7/12
354/354 [==============================] - 10s 27ms/step - loss: 0.8278 - accuracy: 0.7350
Epoch 8/12
354/354 [==============================] - 10s 28ms/step - loss: 0.8170 - accuracy: 0.7415
Epoch 9/12
354/354 [==============================] - 10s 28ms/step - loss: 0.8051 - accuracy: 0.7425
Epoch 10/12
354/354 [==============================] - 10s 28ms/step - loss: 0.7984 -

In [16]:
#anontherone from https://medium.com/x8-the-ai-community/audio-classification-using-cnn-coding-example-f9cbd272269e
#this one isn't working
input_shape=(15,173)
model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.25))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(n_classes, activation='softmax'))

ValueError: Input 0 of layer conv2d is incompatible with the layer: : expected min_ndim=4, found ndim=3. Full shape received: (None, 15, 173)

In [ ]:

opt = keras.optimizers.Adam(learning_rate=0.01)

model3.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
#Train The Model
x_train = np.array(mfcc_train)
y_train = np.array(y_train)
model.fit(x_train, y_train, batch_size=4, epochs=10, verbose=1, validation_data=(x_test, y_test))

In [ ]:
# Testing the model 
x_test = np.array(mfcc_test)
y_test = np.array(y_test)
score = model.evaluate(x_test,y_test)